# 💫 Welcome to the JEDi Tutorial! 💫

First, could you install our package using pip install?

In [1]:
!pip install videojedi==0.1.2

# Get MMNIST data loader.

In [2]:
import torch
def custom_collate(batch):
    import torch
    videos, labels = [], []
    for video in batch:
        videos.append(video)
    return torch.utils.data.dataloader.default_collate(videos), None

def get_MMNIST_loader(train=True, batch_size=20, image_size=(240, 320)):
    from torchvision.datasets import MovingMNIST
    from torch.utils.data import DataLoader
    from torchvision import transforms
    import av
    import torch.nn as nn

    transform = transforms.Compose([
                # TODO: this should be done by a video-level transfrom when PyTorch provides transforms.ToTensor() for video
                # scale in [0, 1] of type float
                transforms.Lambda(lambda x: x / 255.),
                # change from 1 channel to 3
                transforms.Lambda(lambda x: x.repeat(1, 3, 1, 1)),
                # # reshape into (T, C, H, W) for easier convolutions
                # transforms.Lambda(lambda x: x.permute(0, 3, 1, 2)),
                # rescale to the most common size
                transforms.Lambda(lambda x: nn.functional.interpolate(x, image_size)),
    ])
    mmnist = MovingMNIST(".", split="train" if train else "test", download=True, transform=transform)
    return DataLoader(mmnist, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)

train_loader = get_MMNIST_loader(train=True)
test_loader = get_MMNIST_loader(train=False)

# Compute metric.

In [3]:
from videojedi import JEDiMetric
jedi = JEDiMetric(feature_path='.', model_dir='.')
jedi.load_features(train_loader=train_loader, test_loader=test_loader, num_samples=1500)
assert jedi.train_features.shape == (1500, 1280)
assert jedi.test_features.shape == (1500, 1280)

/usr/local/lib/python3.10/dist-packages/videojedi/JEDi.py:11: UserWarning: feature_path is not provided, will not save computed features.
  warnings.warn("feature_path is not provided, will not save computed features.")
INFO:root:loaded params...
INFO:root:SLURM vars not set (distributed training not available)
INFO:root:Initialized (rank/world-size) 0/1
INFO:root:Loading pretrained model from ./vith16.pth.tar
/usr/local/lib/python3.10/dist-packages/videojedi/V_JEPA.py:117: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will

VisionTransformer(
  (patch_embed): PatchEmbed3D(
    (proj): Conv3d(3, 1280, kernel_size=(2, 16, 16), stride=(2, 16, 16))
  )
  (blocks): ModuleList(
    (0-31): 32 x Block(
      (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1280, out_features=3840, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1280, out_features=1280, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
)


/usr/local/lib/python3.10/dist-packages/videojedi/V_JEPA.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(r_path, map_location=device)
INFO:root

Computing features for training set


/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


Saved features to ./train.npy
Computing features for testing set
Saved features to ./test.npy


In [4]:
print(f"JEDi Metric: {jedi.compute_metric()}")

JEDi Metric: 0.00691339373588562


Alternatively, you can load your pre-extracted V-JEPA features using Numpy and directly calculate the distribution distance

In [5]:
from videojedi import JEDiMetric
import numpy as np
jedi = JEDiMetric()
jedi.train_features = np.random.rand(5000, 1280)
jedi.test_features = np.random.rand(5000, 1280)
print(f"JEDi Metric: {jedi.compute_metric()}")

JEDi Metric: 0.001983016839501861
